In [40]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from functools import reduce


from bold_dementia.cogpred import make_training_data, make_all_data, extract_net, MockAtlas
from bold_dementia.utils.configuration import get_config

from neuroginius.atlas import Atlas
from bold_dementia.cogpred import plot_matrix, MatrixResult

config = get_config()
atlas = Atlas.from_name("schaefer200")
gsr = "false"


matrices, metadata = make_all_data(
    config["connectivity_matrices"],
    atlas.name,
    k=3,
    suffix=f"kind-corr_gsr-{gsr}"
)
sns.reset_defaults()

In [41]:
defaults = np.stack([extract_net(mat, atlas, "Default")[0] for mat in matrices])
_, regions = extract_net(matrices[0], atlas, "Default")


In [42]:
plot_matrix(matrices.mean(axis=0), atlas, cmap="seismic")

<Axes: >

In [43]:
comp_lst = []

matrix_res = MatrixResult(defaults, MockAtlas(regions))
for i, sorted_mat in enumerate(matrix_res.sorted_matrices):
    gen = matrix_res._gen_upper_triangular(sorted_mat, lambda x: (x.mean(),))
    comparisons = pd.DataFrame(gen, columns=["node_a", "node_b", "connectivity"])
    comparisons["sub"] = metadata.loc[i, "sub"]
    comparisons["ses"] = metadata.loc[i, "ses"]
    comp_lst.append(comparisons)


df = pd.concat(comp_lst)

In [44]:
comparisons

,node_a,node_b,connectivity,sub,ses
0,PFC,PFCdPFCm,0.259756,2449,M000
1,PFC,PFCv,0.346017,2449,M000
2,PFC,PHC,0.119366,2449,M000
3,PFC,Par,0.284044,2449,M000
4,PFC,Temp,0.202258,2449,M000
5,PFC,pCunPCC,0.265216,2449,M000
6,PFCdPFCm,PFCv,0.314780,2449,M000
7,PFCdPFCm,PHC,0.106363,2449,M000
8,PFCdPFCm,Par,0.316119,2449,M000
9,PFCdPFCm,Temp,0.155917,2449,M000


In [45]:
df.to_csv(f"output/connectivity_export_uniquevals/schaefer-yeo_intra_dmn_gsr-{gsr}.csv")